<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-1-public/blob/adding_C4/C4/W2/ungraded_labs/C4_W2_Lab_2_single_layer_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

**Note:** This notebook can run using TensorFlow 2.5.0

In [2]:
#!pip install tensorflow==2.5.0

In [3]:
%run downloader.ipynb

In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

2.8.0


In [5]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)

def trend(time, slope=0):
    return slope * time

def seasonal_pattern(season_time):
    """Just an arbitrary pattern, you can change it if you wish"""
    return np.where(season_time < 0.4,
                    np.cos(season_time * 2 * np.pi),
                    1 / np.exp(3 * season_time))

def seasonality(time, period, amplitude=1, phase=0):
    """Repeats the same pattern at each period"""
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)

def noise(time, noise_level=1, seed=None):
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level

time = np.arange(4 * 365 + 1, dtype="float32")
baseline = 10
series = trend(time, 0.1)  
baseline = 10
amplitude = 40
slope = 0.05
noise_level = 5

# Create the series
series = baseline + trend(time, slope) + seasonality(time, period=365, amplitude=amplitude)
# Update with noise
series += noise(time, noise_level, seed=42)

split_time = 1000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

window_size = 20
batch_size = 32
shuffle_buffer_size = 1000

In [6]:
df = info_data.reset_index().drop(columns='index')
df.set_index('date',inplace=True)
df = df.astype(float, errors = 'raise')
df

,open,close,high,low,volume,amount
date,,,,,,
1554768000,5241.0,5143.5,5241.0,5080.9,533.910693,2.751033e+06
1554854400,5143.5,5305.7,5400.0,5130.9,484.596751,2.545519e+06
1554940800,5307.6,5016.0,5332.4,4926.4,466.736182,2.370637e+06
1555027200,5018.0,5050.6,5075.0,4873.8,247.758487,1.240250e+06
1555113600,5051.6,5041.7,5091.6,5005.5,167.256504,8.457405e+05
...,...,...,...,...,...,...
1648944000,45806.6,46414.5,47476.0,45534.0,24653.422460,1.143633e+09
1649030400,46414.6,46584.1,46884.6,45080.4,36537.816497,1.679317e+09
1649116800,46584.1,45501.0,47193.0,45358.0,14414.362821,6.666306e+08


In [7]:
df.to_numpy()

array([[5.24100000e+03, 5.14350000e+03, 5.24100000e+03, 5.08090000e+03,
        5.33910693e+02, 2.75103330e+06],
       [5.14350000e+03, 5.30570000e+03, 5.40000000e+03, 5.13090000e+03,
        4.84596751e+02, 2.54551897e+06],
       [5.30760000e+03, 5.01600000e+03, 5.33240000e+03, 4.92640000e+03,
        4.66736182e+02, 2.37063720e+06],
       ...,
       [4.65841000e+04, 4.55010000e+04, 4.71930000e+04, 4.53580000e+04,
        1.44143628e+04, 6.66630594e+08],
       [4.55010000e+04, 4.31668000e+04, 4.55064000e+04, 4.31137000e+04,
        1.86588651e+04, 8.32264035e+08],
       [4.31636000e+04, 4.38253000e+04, 4.38336000e+04, 4.27218000e+04,
        4.29991073e+03, 1.86374725e+08]])

In [22]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
  dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset

In [8]:
dataset = tf.data.Dataset.from_tensor_slices(df)
dataset = dataset.window(5, shift=1, drop_remainder=True)
for window_dataset in dataset:
    for val in window_dataset:
        print(val.numpy(), end=' ')
    print()

[5.24100000e+03 5.14350000e+03 5.24100000e+03 5.08090000e+03
 5.33910693e+02 2.75103330e+06] [5.14350000e+03 5.30570000e+03 5.40000000e+03 5.13090000e+03
 4.84596751e+02 2.54551897e+06] [5.30760000e+03 5.01600000e+03 5.33240000e+03 4.92640000e+03
 4.66736182e+02 2.37063720e+06] [5.01800000e+03 5.05060000e+03 5.07500000e+03 4.87380000e+03
 2.47758487e+02 1.24025049e+06] [5.05160000e+03 5.04170000e+03 5.09160000e+03 5.00550000e+03
 1.67256504e+02 8.45740494e+05] 
[5.14350000e+03 5.30570000e+03 5.40000000e+03 5.13090000e+03
 4.84596751e+02 2.54551897e+06] [5.30760000e+03 5.01600000e+03 5.33240000e+03 4.92640000e+03
 4.66736182e+02 2.37063720e+06] [5.01800000e+03 5.05060000e+03 5.07500000e+03 4.87380000e+03
 2.47758487e+02 1.24025049e+06] [5.05160000e+03 5.04170000e+03 5.09160000e+03 5.00550000e+03
 1.67256504e+02 8.45740494e+05] [5.04580000e+03 5.13370000e+03 5.14890000e+03 5.00250000e+03
 1.76232396e+02 8.92882276e+05] 
[5.30760000e+03 5.01600000e+03 5.33240000e+03 4.92640000e+03
 4.6673

KeyboardInterrupt: 

In [ ]:
dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)
print(dataset)
l0 = tf.keras.layers.Dense(1, input_shape=[window_size])
model = tf.keras.models.Sequential([l0])


model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(learning_rate=1e-6, momentum=0.9))
model.fit(dataset,epochs=100,verbose=0)

print("Layer weights {}".format(l0.get_weights()))


<PrefetchDataset shapes: ((None, None), (None,)), types: (tf.float32, tf.float32)>
Layer weights [array([[-0.03742385],
       [-0.01502325],
       [-0.00724999],
       [ 0.04268194],
       [ 0.02166317],
       [ 0.03091976],
       [ 0.03848791],
       [-0.01195698],
       [-0.08892472],
       [-0.02115656],
       [ 0.07197586],
       [ 0.02514126],
       [-0.1463648 ],
       [ 0.09249936],
       [ 0.05799906],
       [ 0.12946686],
       [-0.04448596],
       [ 0.1218257 ],
       [ 0.26540333],
       [ 0.48541507]], dtype=float32), array([0.01722299], dtype=float32)]


In [ ]:
forecast = []

for time in range(len(series) - window_size):
  forecast.append(model.predict(series[time:time + window_size][np.newaxis]))

forecast = forecast[split_time-window_size:]
results = np.array(forecast)[:, 0, 0]


plt.figure(figsize=(10, 6))

plot_series(time_valid, x_valid)
plot_series(time_valid, results)

In [ ]:
tf.keras.metrics.mean_absolute_error(x_valid, results).numpy()